In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re
import pandas as pd
import matplotlib.pyplot as plt
import math

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: Qt5Agg


# Choose whether absolute or relative pupil size to be used

In [2]:
absoluteSize = False # True means absolute, False means relative

In [3]:
removeNtrials = 2
divideIntoNparts = 4

In [4]:
TimeDwellOrig = 800
TimeFixation = 300

In [10]:
def filterBlinks(pupilData, timeListComplete):
    # filter any blinks and nan values lasting around 250ms (on average)
    # http://faculty.washington.edu/chudler/facts.html
   
    
   # Blink detection    
    missingPupilData = dict()
    missingPupilData['Start'] = list()
    missingPupilData['End'] = list()
    missingPupilData['Duration'] = list()
    missingPupilData['DurationExtra'] = list()
    
    pupilData_filter = list()
    time_filter = list()
    index_blinkEnd = list()
    
    missingBeginning = 0
    missingEnd = 0

    # First, find the missing values
    pupilInd = -1
    for pupilD in pupilData:
        
        pupilInd = pupilInd + 1
        #if pupilInd>772 and pupilInd<940:
        #    print(pupilInd, pupilD)
            
        # check if val is nan
        if np.isnan(pupilD):
            # check if first element of list
            if pupilInd == 0:
                # first value is nan
                missingPupilData['Start'].append(pupilInd)
                #print('Start', missingPupilData['Start'][-1])
                
            # check if previous val was nan
            elif np.isnan(pupilData[pupilInd-1]):
                
                # check if val is not last element
                if pupilInd < len(pupilData)-1:
                    # check if next val is not nan
                    if not np.isnan(pupilData[pupilInd+1]):
                        # note the index for end of blink
                        missingPupilData['End'].append(pupilInd)
                        #print('End', missingPupilData['End'][-1])
                else:
                    # last value is nan
                    missingEnd = 1
                    missingPupilData['End'].append(pupilInd)
                    #print('End', missingPupilData['End'][-1])
                    
            # if new nan, note the index
            else:
                missingPupilData['Start'].append(pupilInd)
                #print('Start', missingPupilData['Start'][-1])
                
                # for single nan values
                # check if val is not last element
                if pupilInd < len(pupilData)-1:
                    # check if next val is not nan
                    if not np.isnan(pupilData[pupilInd+1]):
                        # note the index for end of blink
                        missingPupilData['End'].append(pupilInd)
                        #print('End', missingPupilData['End'][-1])
                else:
                    # last value is nan
                    missingPupilData['End'].append(pupilInd)
                    #print('End', missingPupilData['End'][-1])
            
    
    
    # Second, create a list that does not contain missing values or 200ms before and after that
    
     # blink is every nan value in the range of 100-400ms 
    # 200 ms (18 samples) before and after the blink will also be removed
    addedBlinkSamples = 18    
    blinkDurationStart = 0
    blinkDurationEnd = 0
    
    iCurrent = len(missingPupilData['Start'])
    endCurrent = iCurrent - 1
    isBlink = 0
    pupilData_filter = list(pupilData)
    time_filter = list(timeListComplete)
    
    index_blinkEndExtra = list() # list to document if 2 blinks joined 
    
    while iCurrent > 0:
        
        iCurrent = iCurrent - 1
        
        #print('Current', missingPupilData['Start'][iCurrent], missingPupilData['End'][iCurrent])
        
        # check if it is really a blink, and remove if it is not
        # if blink duration is less than 100ms, the reason for missing data is something else
        timeStart0 = time_filter[missingPupilData['Start'][iCurrent]].partition('+')
        timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
        timeEnd0 = time_filter[missingPupilData['End'][iCurrent]].partition('+')
        timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
        time1 = (timeEnd - timeStart).total_seconds()
        #print(time1)
        if time1 < 0.1 or time1 > 1.5:
            isNotBlink = 1
        else:
            isNotBlink = 0
            
        # Remove if it is a blink
        if isNotBlink:
            del pupilData_filter[missingPupilData['Start'][iCurrent]:missingPupilData['End'][iCurrent]+1]
            del time_filter[missingPupilData['Start'][iCurrent]:missingPupilData['End'][iCurrent]+1]
            
            # length of non-blink
            lengthBlink = missingPupilData['End'][iCurrent] - missingPupilData['Start'][iCurrent] + 1
            
            # all blinks after this non-blink need to be reduced in indexes
            #print(len(missingPupilData['Start'])-1)
            
            for iChange in range(iCurrent-1, len(missingPupilData['Start'])-1):
                #print('i', iChange)
                if iChange == iCurrent-1:
                    
                    #print('Deleted', missingPupilData['Start'][iChange+1], missingPupilData['End'][iChange+1])
                    #print(lengthBlink)
                    del missingPupilData['Start'][iChange+1]
                    del missingPupilData['End'][iChange+1]
                else:
                    
                    #print('Old', missingPupilData['Start'][iChange], missingPupilData['End'][iChange])
                    missingPupilData['Start'][iChange] = missingPupilData['Start'][iChange] - lengthBlink
                    missingPupilData['End'][iChange] = missingPupilData['End'][iChange] - lengthBlink
                    #print('New', missingPupilData['Start'][iChange], missingPupilData['End'][iChange])
                    
    #print(len(missingPupilData['Start']))
    
    previousBlinkEnd = 0
    nextBlinkEnd = 0
    iCurrent = len(missingPupilData['Start'])
    
#     for index, (value1, value2) in enumerate(zip(missingPupilData['Start'], missingPupilData['End'])):
#         print(index, value1, value2)
#     for i in enumerate(pupilData_filter):
#         print(i)
    
    while iCurrent > 0:
        
        iCurrent = iCurrent - 1
        #print(iCurrent)
        iNow = iCurrent # iCurrent is modified in advance later (possibly), which will disturb the current data index
        #print(missingPupilData['Start'][iNow], missingPupilData['End'][iNow])
        
#         if iNow > 1 :
#             print('BEFORE!!!!!!!!!!!!!!!!!')
#             print(pupilData_filter[missingPupilData['Start'][iNow]:-1])
            
        # to make sure that blink removal data does not go into the previous data
        if iCurrent == 0:
            previousBlinkEnd = 0
        else:
            previousBlinkEnd = missingPupilData['Start'][iCurrent-1]

        # to make sure blink does not go into the next one
        if iCurrent == len(missingPupilData['Start'])-1:
            #print(iCurrent)
            nextBlinkEnd = len(pupilData_filter)
            #print(nextBlinkEnd)
        else:
            nextBlinkEnd = missingPupilData['Start'][iCurrent+1]
            
        
        
        if missingPupilData['End'][iCurrent] + addedBlinkSamples > nextBlinkEnd:
            #LOOK INTO THIS, NEXT BLINK IS REMOVED BUT ITS EXTRA 200MS IS NOT!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            #print('After', missingPupilData['End'][iCurrent], addedBlinkSamples, '>>>>>', nextBlinkEnd)
            blinkRemoveAfter = abs(missingPupilData['End'][iCurrent] - nextBlinkEnd)
        else:
            blinkRemoveAfter = addedBlinkSamples
        
        if missingPupilData['Start'][iCurrent] - addedBlinkSamples < previousBlinkEnd:
            #print('Before', missingPupilData['Start'][iCurrent], addedBlinkSamples, '<<<<<<', previousBlinkEnd)
            blinkRemoveBefore = abs(missingPupilData['Start'][iCurrent] - previousBlinkEnd)
            
            # Add the extra duration to list
            if iCurrent > 0:
                
                timeStart0 = time_filter[missingPupilData['Start'][iCurrent-1]].partition('+')
                timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                timeEnd0 = time_filter[missingPupilData['End'][iCurrent-1]].partition('+')
                timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                time1 = (timeEnd - timeStart).total_seconds()
                #print(timeEnd, timeStart, time1)

                missingPupilData['DurationExtra'].append(time1)
#             if iCurrent > 1:
#                 print(missingPupilData['Start'][iCurrent-1], 'also removed')
#             # After next blink is also removed, 
            iCurrent = iCurrent - 1
            index_blinkEndExtra.append(2)
            
                
        else:
            blinkRemoveBefore = addedBlinkSamples
            index_blinkEndExtra.append(1)
            missingPupilData['DurationExtra'].append(0)
            
#         print(missingPupilData['Start'][iNow]-blinkRemoveBefore)
        
        timeStart0 = time_filter[missingPupilData['Start'][iNow]].partition('+')
        timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        #print(timeStart, pupilData_filter[missingPupilData['Start'][iNow]])     
        timeEnd0 = time_filter[missingPupilData['End'][iNow]].partition('+')
        timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        #print(timeEnd) 
        time2 = (timeEnd - timeStart).total_seconds()
        #print(time2)
        missingPupilData['Duration'].append(time2)
        del pupilData_filter[missingPupilData['Start'][iNow]-blinkRemoveBefore:missingPupilData['End'][iNow]+blinkRemoveAfter]
        del time_filter[missingPupilData['Start'][iNow]-blinkRemoveBefore:missingPupilData['End'][iNow]+blinkRemoveAfter]
        
        index_blinkEnd.append(missingPupilData['Start'][iNow] - blinkRemoveBefore - 1)
    #         if iNow > 1 :
#             print('AFTER!!!!!!!!!!!!')
#             if np.nan in pupilData_filter[missingPupilData['Start'][iNow]:-1]:
#                 for i,j in enumerate(pupilData_filter[missingPupilData['Start'][iNow]:-1]):
#                     print(i+missingPupilData['End'][iNow]+blinkRemoveAfter,j)
        
    
    if np.nan in pupilData_filter:
        for i in enumerate(pupilData_filter):
            print(i)
     
    index_blinkEnd = list(reversed(index_blinkEnd))
    
    return pupilData_filter, time_filter, index_blinkEnd, index_blinkEndExtra, missingPupilData

In [11]:
# Create list of list (made of epochs) composed of times when user looked at scratchPad

def UserLookedAtNewTrial(UserLooksAtKey):
    # Create list of list (made of epochs) composed of times when user looked at NextPhrase
    
    epoch = list()   
    epochList = list()
    
    nRow = -1
    firstNextPhrase = 0
    flagStart = 0
    firstTrial = 0
    
    # Find unique looks at new trial, by checking if progress pct is greater than previous one and store first look in a list
    
    progressNextPhrase = 0 
    
    for row in UserLooksAtKey:
        nRow = nRow + 1
        
        # first add data for first trial, before first next phrase is reached
        if firstTrial == 0:
            firstTrial = 1
            epochList.append([row, 0])
            ##print(epochList)
            continue
            
        if row[1] == 'NextPhrase':
            
            # first eliminate an ongoing look
            if UserLooksAtKey[nRow-1][1] == 'NextPhrase':
                continue
            else:
                # normally for new looks
                
                # if it is a continued look, progressNextPhrase should not be 0
                ##print('Continued ???', progressNextPhrase, UserLooksAtKey[nRow])
                
                if progressNextPhrase > float(UserLooksAtKey[nRow][2]):
                    
                    # save it as 0, if not a continued look
                    ##print('progress is now 0')
                    progressNextPhrase = 0
#                 else:
#                     if progressNextPhrase == float(UserLooksAtKey[nRow][2]):
#                         #print('ITS EQUAL!!!')
                    
                
            # now check if this look is going to be successfully selected:
            nextPhrase_selected = 0
            rowCurrent = nRow-1
            
            while nextPhrase_selected < 1:
                rowCurrent = rowCurrent + 1
                if UserLooksAtKey[rowCurrent][1] == 'NextPhrase':
                    if float(UserLooksAtKey[rowCurrent][2]) == 1:
                        ##print(UserLooksAtKey[rowCurrent])
                        nextPhrase_selected = 1
                        rowCurrentEnd = rowCurrent
                        
                        
                        if progressNextPhrase != 0:
                            epochList[-1].insert(2, UserLooksAtKey[rowPreviousEnd])
                            epochList.append([UserLooksAtKey[rowCurrentStart], UserLooksAtKey[rowCurrentEnd]])
                        else:
                            epochList[-1].insert(2, UserLooksAtKey[nRow-1])
                            epochList.append([UserLooksAtKey[nRow], UserLooksAtKey[rowCurrentEnd]])
                    else:
                        continue
                else:
                    # if look at next phrase is stopped, store the previous look progress, in case it is continued later
                    progressNextPhrase = float(UserLooksAtKey[rowCurrent-1][2])
                    rowCurrentStart = nRow
                    rowPreviousEnd = nRow - 1
                    break
                    
                     
                    
                    
    # insert last element before quiting - 
    # keys looked at list
    KeysLookedAtFromUserKeys = [key[1] for key in UserLooksAtKey]
    
    # find key before quit key
    nBegin = 300
    foundQuitKey = 0
    indBeforeQuit = 0
    while foundQuitKey < 1:
        if KeysLookedAtFromUserKeys[-nBegin] == 'Quit':
            # look in direction of previous keys
            # if previous key is not quit, you can stop
            if KeysLookedAtFromUserKeys[-nBegin-1] != 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin - 1
                break
            else:
                nBegin = nBegin + 1
        else:
            # current key is not quit, start looking in the other direction
            if KeysLookedAtFromUserKeys[-nBegin + 1] == 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin
                break
            else:
                nBegin = nBegin - 1
    
    epochList[-1].insert(2, UserLooksAtKey[indBeforeQuit])
    
    iRemove = removeNtrials
    while iRemove > 0:
        iRemove = iRemove - 1
        epochList.remove(epochList[iRemove])
    
#     for i in epochList:
#         print(i)
#     print('next')
    return epochList[0][0][0]

In [5]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        if key[1] == 'IncreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [6]:
# Create list of list (made of epochs) composed of times when user looked at scratchPad

def UserLookedAtNewTrial(UserLooksAtKey):
    # Create list of list (made of epochs) composed of times when user looked at NextPhrase
    
    epoch = list()   
    epochList = list()
    
    nRow = -1
    firstNextPhrase = 0
    flagStart = 0
    firstTrial = 0
    
    # Find unique looks at new trial, by checking if progress pct is greater than previous one and store first look in a list
    
    progressNextPhrase = 0 
    
    for row in UserLooksAtKey:
        nRow = nRow + 1
        
        # first add data for first trial, before first next phrase is reached
        if firstTrial == 0:
            firstTrial = 1
            epochList.append([row, 0])
            ##print(epochList)
            continue
            
        if row[1] == 'NextPhrase':
            
            # first eliminate an ongoing look
            if UserLooksAtKey[nRow-1][1] == 'NextPhrase':
                continue
            else:
                # normally for new looks
                
                # if it is a continued look, progressNextPhrase should not be 0
                ##print('Continued ???', progressNextPhrase, UserLooksAtKey[nRow])
                
                if progressNextPhrase > float(UserLooksAtKey[nRow][2]):
                    
                    # save it as 0, if not a continued look
                    ##print('progress is now 0')
                    progressNextPhrase = 0
#                 else:
#                     if progressNextPhrase == float(UserLooksAtKey[nRow][2]):
#                         #print('ITS EQUAL!!!')
                    
                
            # now check if this look is going to be successfully selected:
            nextPhrase_selected = 0
            rowCurrent = nRow-1
            
            while nextPhrase_selected < 1:
                rowCurrent = rowCurrent + 1
                if UserLooksAtKey[rowCurrent][1] == 'NextPhrase':
                    if float(UserLooksAtKey[rowCurrent][2]) == 1:
                        ##print(UserLooksAtKey[rowCurrent])
                        nextPhrase_selected = 1
                        rowCurrentEnd = rowCurrent
                        
                        
                        if progressNextPhrase != 0:
                            epochList[-1].insert(2, UserLooksAtKey[rowPreviousEnd])
                            epochList.append([UserLooksAtKey[rowCurrentStart], UserLooksAtKey[rowCurrentEnd]])
                        else:
                            epochList[-1].insert(2, UserLooksAtKey[nRow-1])
                            epochList.append([UserLooksAtKey[nRow], UserLooksAtKey[rowCurrentEnd]])
                    else:
                        continue
                else:
                    # if look at next phrase is stopped, store the previous look progress, in case it is continued later
                    progressNextPhrase = float(UserLooksAtKey[rowCurrent-1][2])
                    rowCurrentStart = nRow
                    rowPreviousEnd = nRow - 1
                    break
                    
                     
                    
                    
    # insert last element before quiting - 
    # keys looked at list
    KeysLookedAtFromUserKeys = [key[1] for key in UserLooksAtKey]
    
    # find key before quit key
    nBegin = 300
    foundQuitKey = 0
    indBeforeQuit = 0
    while foundQuitKey < 1:
        if KeysLookedAtFromUserKeys[-nBegin] == 'Quit':
            # look in direction of previous keys
            # if previous key is not quit, you can stop
            if KeysLookedAtFromUserKeys[-nBegin-1] != 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin - 1
                break
            else:
                nBegin = nBegin + 1
        else:
            # current key is not quit, start looking in the other direction
            if KeysLookedAtFromUserKeys[-nBegin + 1] == 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin
                break
            else:
                nBegin = nBegin - 1
    
    epochList[-1].insert(2, UserLooksAtKey[indBeforeQuit])
    
    iRemove = removeNtrials
    while iRemove > 0:
        iRemove = iRemove - 1
        epochList.remove(epochList[iRemove])
    
#     for i in epochList:
#         print(i)
#     print('next')
    return epochList[0][0][0]

In [12]:
# Create list of list (made of epochs) composed of times when user looked at scratchPad

def UserLookedAtScratchPad(UserLooksAtKey):
    epoch = list()   
    epochList = list()
    
    nRow = -1
    firstScratchPad = 0
    flagStart = 0
    
    
    # Find unique looks at scratchpad, by checking if progress pct is greater than previous one and store first look in a list
    
    for row in UserLooksAtKey:
        nRow = nRow + 1
        
        if row[1] == 'ScratchPad':
            
            # for consecutive scratchpad looks
            if UserLooksAtKey[nRow-1][1] == 'ScratchPad':
                if float(row[2]) == 0:
                    progressScratchPad = float(UserLooksAtKey[nRow-1][2])
                else:
                    progressScratchPad = float(row[2])
    
            # if first time of the look at scratchpad
            else:
                # if not the very first look (means the variable for progress exists)
                if firstScratchPad > 0: 
                    # if the current progress value is less than the previous one, means it is a new look 
                    if progressScratchPad > float(row[2]):
                        # if progress is not 0, then update the progress --> sometimes, it is 0 at the end of the look
                        if float(row[2]) != 0:
                            progressScratchPad = float(row[2])
                            epochList.append(row)
                            #print(row)
                            
                # if the very first look (means the variable for progress does not exist)    
                else:
                    progressScratchPad = float(row[2])
                    firstScratchPad = 1
                    epochList.append(row)
                    #print(row)

    return epochList

In [7]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [8]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        print('no nearest time')
        
    return nearestTP, nearestTPind

In [15]:
# find end time of Expt - when the scratchpad is erased, at the end
def FindTimeEnd(scratchPadList):
    
    scratchPadText = [i[1] for i in scratchPadList]
    scratchPadTime = [i[0] for i in scratchPadList]
                
    iText = len(scratchPadText)
    while iText > 0:
        iText = iText - 1
        if scratchPadText[iText] == '':
            if scratchPadText[iText-1] != '':
                findTimeAtIndex = iText
                break
                    
    return scratchPadTime[findTimeAtIndex]

In [9]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #plt.plot(vals_orig)
    #Make copy so original not edited
    vals1 = pd.DataFrame(vals_orig)      
    vals0 = vals1.replace([np.inf, -np.inf], np.nan)
    vals = vals0.astype(float).interpolate('linear', limit_direction = 'both') # linear interpolation instead of 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data, limit direction
    # set to both, to interpolate the nan values occuring from the start of the series
    
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    
    #print(rolling_median)
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx]
    #vals.plot()
    return(vals)

In [25]:
def computeAggregateAverage(scratchPadLookedAtEpoch, timeTrial3Starts, timeExptEnds, timeGazeLog, pupilLogL_wFilter, pupilLogR_wFilter, GazeLog, subjName):
    
    typingMechanism = subjName[-2:]
    rowTimeList = -1
    timeUserLooked = list()
    timeScratchAll = list()
    timeToBeScratchedAll = list()
   
    phraseScratchedInd = 0
    phraseToBeScratchedInd = 0
    
    # create a list of times 
    timeStrUserLooked =  [item[0] for item in scratchPadLookedAtEpoch]
    
    # convert the list of strings to datetime formats
    timeUserLooked = timeConversion(timeStrUserLooked)
        
    winSize = 40
    pupilLogL = hampel(pupilLogL_wFilter, winSize, 3)
    pupilLogR = hampel(pupilLogR_wFilter, winSize, 3)

    # Pupil distance (x,y,z) in User Coordinate system 
    pupilDistLx = [float(item6[3]) if 'Invalid' not in item6 else np.nan for item6 in GazeLog]
    pupilDistLy = [float(item7[4]) if 'Invalid' not in item7 else np.nan for item7 in GazeLog]
    pupilDistLz = [float(item8[5]) if 'Invalid' not in item8 else np.nan for item8 in GazeLog]
    pupilDistRx = [float(item9[10]) if 'Invalid' not in item9 else np.nan for item9 in GazeLog]
    pupilDistRy = [float(item10[11]) if 'Invalid' not in item10 else np.nan for item10 in GazeLog]
    pupilDistRz = [float(item11[12]) if 'Invalid' not in item11 else np.nan for item11 in GazeLog]
    
    # create dataframe from the pupil distances
    df_pupilDist = pd.DataFrame(np.column_stack([pupilDistLx, pupilDistLy, pupilDistLz, pupilDistRx, pupilDistRy, pupilDistRz]), columns=['pupilDistLx', 'pupilDistLy', 'pupilDistLz', 'pupilDistRx', 'pupilDistRy', 'pupilDistRz'])
    df_pupilDist = df_pupilDist.interpolate('linear', limit_direction = 'both')
    #print(df_pupilDist.pupilDistLx)
    
    pupilDist = [math.sqrt((df_pupilDist.pupilDistLx[indPt]-df_pupilDist.pupilDistRx[indPt])**2 + (df_pupilDist.pupilDistLy[indPt]-df_pupilDist.pupilDistRy[indPt])**2 + (df_pupilDist.pupilDistLz[indPt]-df_pupilDist.pupilDistRz[indPt])**2) for indPt in range(0, len(pupilDistLx))]
    
    #print(timeUserLooked)
    
    for timeList in timeUserLooked:
        
        # restrcitions on timeList: If it occurs in the first 2 trials or after the expt ends, continue
        if timeList < timeTrial3Starts or timeList > timeExptEnds:
            #print(timeList, timeTrial3Starts, timeExptEnds)
            continue
        
        rowTimeList = rowTimeList + 1
        # Extract epoch data from gazelog
        
        # from the start of look at scratchpad, remove the dwell time passed and fixation time, to get the true starting time 
        timeScratchPadActive = timeList - datetime.timedelta(milliseconds=float(scratchPadLookedAtEpoch[rowTimeList][3])) - datetime.timedelta(milliseconds=TimeFixation)
#        timeScratchPadActive = timeList - datetime.timedelta(milliseconds=float(scratchPadLookedAtEpoch[rowTimeList][3]))

        timeGazeLogStart, GazeLogStartInd = nearestTimePoint(timeGazeLog, timeScratchPadActive)
        
        timeWindowEnd = timeList + datetime.timedelta(seconds=10)
        
        timeGazeLogEnd, GazeLogEndInd = nearestTimePoint(timeGazeLog, timeWindowEnd)
        #print(GazeLogStartInd, GazeLogEndInd)
        
        if GazeLogStartInd != GazeLogEndInd:
            pupilWoOutlierL = pupilLogL[GazeLogStartInd:GazeLogEndInd]
            pupilWoOutlierR = pupilLogR[GazeLogStartInd:GazeLogEndInd]
            interPupilDist = pupilDist[GazeLogStartInd:GazeLogEndInd]
            
            winSize = 40
            # Filter pupil sizes
            #pupilWoOutlierL = hampel(gazeEpochL, winSize, 3)
            #pupilWoOutlierR = hampel(gazeEpochR, winSize, 3)
            
            # Does pupilWoOutlier still have nan values -- happens in subject 'lr', the complete list is made of nan values
            if pupilWoOutlierL.isnull().values.any() == True or pupilWoOutlierR.isnull().values.any() == True:
                # This means that the complete list was already only nan values, and cannot be changed, whatsoever
                continue               
            
            # compute correlation between right and left eye pupil sizes
            pupilCorr = pupilWoOutlierL.corrwith(pupilWoOutlierR, axis = 0)
            if pupilCorr.values[0] < 0.8: # 0.8 is a good value for correlation and was also the mean of mean of the correlations
            # for the trials for the users
                continue
 
            # Moving Mean of data without outliers: 
            pupilAbsoluteL = pupilWoOutlierL.rolling(window=winSize, min_periods=1, center=True).mean()
            pupilAbsoluteR = pupilWoOutlierR.rolling(window=winSize, min_periods=1, center=True).mean()
            
            #print(subjName, len(pupilWoOutlierL), len(pupilAbsoluteL))
            
            # Reference of 300ms, and not only the first observation point
            pupilReferenceL = pupilAbsoluteL[0][0:29].mean()
            pupilReferenceR = pupilAbsoluteR[0][0:29].mean()
            
            pupilRelativeL = (pupilAbsoluteL - pupilReferenceL)/pupilReferenceL
            pupilRelativeR = (pupilAbsoluteR - pupilReferenceR)/pupilReferenceR
            
            #pupilRelativeL = (pupilAbsoluteL - pupilAbsoluteL[0][0])/pupilAbsoluteL[0][0]
            #pupilRelativeR = (pupilAbsoluteR - pupilAbsoluteR[0][0])/pupilAbsoluteR[0][0]
            
        else:
            print('one')
            continue
            #continue # remove samples with only 1 sample
        
        #print(pupilRelativeL)
        # Also to check if phrase typed is DT or not, only check the last element of epoch
        #print(timeList[-1])
        if typingMechanism == 'DT':
            # add the pupil size
            if pupilData['DTFirst'] == 0:
                pupilData['DTFirst'] = 1
                
                if absoluteSize:
                    pupilData['DTLeft'] = [pupilAbsoluteL.values[i][0] for i in range(0, len(pupilAbsoluteL.values))]
                    pupilData['DTRight'] = [pupilAbsoluteR.values[i][0] for i in range(0, len(pupilAbsoluteR.values))]
                    
                    pupilData['DTLeftSquared'] = [(pupilAbsoluteL.values[i][0])**2 for i in range(0, len(pupilAbsoluteL.values))]
                    pupilData['DTRightSquared'] = [(pupilAbsoluteR.values[i][0])**2 for i in range(0, len(pupilAbsoluteR.values))]
                    
                else:
                    pupilData['DTLeft'] = [pupilRelativeL.values[i][0] for i in range(0, len(pupilRelativeL.values))]
                    pupilData['DTRight'] = [pupilRelativeR.values[i][0] for i in range(0, len(pupilRelativeR.values))]
                
                    pupilData['DTLeftSquared'] = [(pupilRelativeL.values[i][0])**2 for i in range(0, len(pupilRelativeL.values))]
                    pupilData['DTRightSquared'] = [(pupilRelativeR.values[i][0])**2 for i in range(0, len(pupilRelativeR.values))]
                    
                pupilData['DTNumber'] = pupilData['DTNumber'] + 1
                
                pupilData['DTLeftMax'] = max(pupilData['DTLeft'])
                pupilData['DTRightMax'] = max(pupilData['DTRight'])
                pupilData['DTLeftMin'] = min(pupilData['DTLeft'])
                pupilData['DTRightMin'] = min(pupilData['DTRight'])
                #print(pupilData['DTNumber'])
                
            else:
                if absoluteSize:
                    
                    pupilAddL = [pupilData['DTLeft'][i]+pupilAbsoluteL.values[i][0] for i in range(0, min(len(pupilAbsoluteL[pupilAbsoluteL.columns[0]]), len(pupilData['DTLeft'])))]
                    pupilAddR = [pupilData['DTRight'][i]+pupilAbsoluteR.values[i][0] for i in range(0, min(len(pupilAbsoluteR[pupilAbsoluteR.columns[0]]), len(pupilData['DTRight'])))]
                    
                    pupilAddSquareL = [pupilData['DTLeftSquared'][i]+(pupilAbsoluteL.values[i][0])**2 for i in range(0, min(len(pupilAbsoluteL[pupilAbsoluteL.columns[0]]), len(pupilData['DTLeftSquared'])))]
                    pupilAddSquareR = [pupilData['DTRightSquared'][i]+(pupilAbsoluteR.values[i][0])**2 for i in range(0, min(len(pupilAbsoluteR[pupilAbsoluteR.columns[0]]), len(pupilData['DTRightSquared'])))]
                    
                    pupilData['DTLeftMax'] = max([max(pupilAbsoluteL.values), pupilData['DTLeftMax']])
                    pupilData['DTRightMax'] = max([max(pupilAbsoluteR.values), pupilData['DTRightMax']])
                    pupilData['DTLeftMin'] = min([min(pupilAbsoluteL.values), pupilData['DTLeftMin']])
                    pupilData['DTRightMin'] = min([min(pupilAbsoluteR.values), pupilData['DTRightMin']])
                
                else:
                    pupilAddL = [pupilData['DTLeft'][i]+pupilRelativeL.values[i][0] for i in range(0, min(len(pupilRelativeL[pupilRelativeL.columns[0]]), len(pupilData['DTLeft'])))]
                    pupilAddR = [pupilData['DTRight'][i]+pupilRelativeR.values[i][0] for i in range(0, min(len(pupilRelativeR[pupilRelativeR.columns[0]]), len(pupilData['DTRight'])))]
                    
                    pupilAddSquareL = [pupilData['DTLeftSquared'][i]+(pupilRelativeL.values[i][0])**2 for i in range(0, min(len(pupilRelativeL[pupilRelativeL.columns[0]]), len(pupilData['DTLeftSquared'])))]
                    pupilAddSquareR = [pupilData['DTRightSquared'][i]+(pupilRelativeR.values[i][0])**2 for i in range(0, min(len(pupilRelativeR[pupilRelativeR.columns[0]]), len(pupilData['DTRightSquared'])))]
                    
                    pupilData['DTLeftMax'] = max([max(pupilRelativeL.values), pupilData['DTLeftMax']])
                    pupilData['DTRightMax'] = max([max(pupilRelativeR.values), pupilData['DTRightMax']])
                    pupilData['DTLeftMin'] = min([min(pupilRelativeL.values), pupilData['DTLeftMin']])
                    pupilData['DTRightMin'] = min([min(pupilRelativeR.values), pupilData['DTRightMin']])
                
                pupilData['DTLeft'] = pupilAddL
                pupilData['DTRight'] = pupilAddR
                pupilData['DTLeftSquared'] = pupilAddSquareL
                pupilData['DTRightSquared'] = pupilAddSquareR
                pupilData['DTNumber'] = pupilData['DTNumber'] + 1
                
                
                
        else:
            
            if pupilData['MSFirst'] == 0:
                pupilData['MSFirst'] = 1
                if absoluteSize:
                    
                    pupilData['MSLeft'] = [pupilAbsoluteL.values[i][0] for i in range(0, len(pupilAbsoluteL.values))]
                    pupilData['MSRight'] = [pupilAbsoluteR.values[i][0] for i in range(0, len(pupilAbsoluteR.values))]
                    
                    pupilData['MSLeftSquared'] = [(pupilAbsoluteL.values[i][0])**2 for i in range(0, len(pupilAbsoluteL.values))]
                    pupilData['MSRightSquared'] = [(pupilAbsoluteR.values[i][0])**2 for i in range(0, len(pupilAbsoluteR.values))]
                    
                else:
                    pupilData['MSLeft'] = [pupilRelativeL.values[i][0] for i in range(0, len(pupilRelativeL.values))]
                    pupilData['MSRight'] = [pupilRelativeR.values[i][0] for i in range(0, len(pupilRelativeR.values))]
                    
                    pupilData['MSLeftSquared'] = [(pupilRelativeL.values[i][0])**2 for i in range(0, len(pupilRelativeL.values))]
                    pupilData['MSRightSquared'] = [(pupilRelativeR.values[i][0])**2 for i in range(0, len(pupilRelativeR.values))]
                    
                
                pupilData['MSNumber'] = pupilData['MSNumber'] + 1
                
                pupilData['MSLeftMax'] = max(pupilData['MSLeft'])
                pupilData['MSRightMax'] = max(pupilData['MSRight'])
                pupilData['MSLeftMin'] = min(pupilData['MSLeft'])
                pupilData['MSRightMin'] = min(pupilData['MSRight'])
                
                #print(subjName, 'MS')

            else:
                if absoluteSize:
                    pupilAddL = [pupilData['MSLeft'][i]+pupilAbsoluteL.values[i][0] for i in range(0, min(len(pupilAbsoluteL[pupilAbsoluteL.columns[0]]), len(pupilData['MSLeft'])))]
                    pupilAddR = [pupilData['MSRight'][i]+pupilAbsoluteR.values[i][0] for i in range(0, min(len(pupilAbsoluteR[pupilAbsoluteR.columns[0]]), len(pupilData['MSRight'])))]
                
                    pupilAddSquareL = [pupilData['MSLeft'][i]+(pupilAbsoluteL.values[i][0])**2 for i in range(0, min(len(pupilAbsoluteL[pupilAbsoluteL.columns[0]]), len(pupilData['MSLeftSquared'])))]
                    pupilAddSquareR = [pupilData['MSRight'][i]+(pupilAbsoluteR.values[i][0])**2 for i in range(0, min(len(pupilAbsoluteR[pupilAbsoluteR.columns[0]]), len(pupilData['MSRightSquared'])))]
                    
                    pupilData['MSLeftMax'] = max([max(pupilAbsoluteL.values), pupilData['MSLeftMax']])
                    pupilData['MSRightMax'] = max([max(pupilAbsoluteR.values), pupilData['MSRightMax']])
                    pupilData['MSLeftMin'] = min([min(pupilAbsoluteL.values), pupilData['MSLeftMin']])
                    pupilData['MSRightMin'] = min([min(pupilAbsoluteR.values), pupilData['MSRightMin']])
                    
                else:
                    pupilAddL = [pupilData['MSLeft'][i]+pupilRelativeL.values[i][0] for i in range(0, min(len(pupilRelativeL[pupilRelativeL.columns[0]]), len(pupilData['MSLeft'])))]
                    pupilAddR = [pupilData['MSRight'][i]+pupilRelativeR.values[i][0] for i in range(0, min(len(pupilRelativeR[pupilRelativeR.columns[0]]), len(pupilData['MSRight'])))]
                    
                    pupilAddSquareL = [pupilData['MSLeftSquared'][i]+(pupilRelativeL.values[i][0])**2 for i in range(0, min(len(pupilRelativeL[pupilRelativeL.columns[0]]), len(pupilData['MSLeft'])))]
                    pupilAddSquareR = [pupilData['MSRightSquared'][i]+(pupilRelativeR.values[i][0])**2 for i in range(0, min(len(pupilRelativeR[pupilRelativeR.columns[0]]), len(pupilData['MSRight'])))]
                    
                    pupilData['MSLeftMax'] = max([max(pupilRelativeL.values), pupilData['MSLeftMax']])
                    pupilData['MSRightMax'] = max([max(pupilRelativeR.values), pupilData['MSRightMax']])
                    pupilData['MSLeftMin'] = min([min(pupilRelativeL.values), pupilData['MSLeftMin']])
                    pupilData['MSRightMin'] = min([min(pupilRelativeR.values), pupilData['MSRightMin']])                    
                    
                pupilData['MSLeft'] = pupilAddL
                pupilData['MSRight'] = pupilAddR
                pupilData['MSLeftSquared'] = pupilAddSquareL
                pupilData['MSRightSquared'] = pupilAddSquareR
                pupilData['MSNumber'] = pupilData['MSNumber'] + 1
                

                
    return pupilData

In [26]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
flagFirstSubj = 0
pupilData = dict()
pupilData['DTFirst'] = 0
pupilData['MSFirst'] = 0
pupilData['DTNumber'] = 0
pupilData['MSNumber'] = 0

for root, dirs, subfolder in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    if not dirs:
        
        if 'notCompleted' in root or 'notInclude' in root or 'notEnough' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        userKeys = None
        scratchPad = None
        gazeLog = None
        stimPhrase = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'user_look*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'ScratchPad*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)  
                    scratchPad.remove(scratchPad[0])
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'PhraseLog*'):
                try:
                    fStimPhrase = open(root + '\\' + file, encoding='utf-8')
                    readerStimPhrase = csv.reader(fStimPhrase)
                    stimPhrase = list(readerStimPhrase)
                    stimPhrase.remove(stimPhrase[0])
                except:
                    if fStimPhrase is not None:
                        fStimPhrase.close()
                    else:
                        print('error in opening the phrase log file')
            elif fnmatch.fnmatch(file, 'GazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or scratchPad is None or stimPhrase is None or gazeLog is None:
                continue
            else:
                
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                subjName = a.findall(root)[0]
                typing_mechanism = subjName[-2:]
                print(subjName)
                
                # obtain right and left pupil data
                # Create list of pupil sizes from gazelog
                pupilLogL = [float(item4[29]) if 'Invalid' not in item4 else np.nan for item4 in gazeLog]
                pupilLogR = [float(item5[31]) if 'Invalid' not in item5 else np.nan for item5 in gazeLog]
                timeList = [item[0] for item in gazeLog]
                
                # filter the blinks
                pupilLogL_filter, time_filter, index_blinkEndL, index_blinkEndExtra, missingDataL = filterBlinks(pupilLogL, timeList)
                
                pupilLogR_filter, time_filter, index_blinkEndR, index_blinkEndExtra, missingDataR = filterBlinks(pupilLogR, timeList)
                
                time_filterFormatted = timeConversion(time_filter)
                
                # Compute dwell time
                userKeysWTime = ComputeDwellTime(userKeys)

                # call function to check when scratchpad is looked at and save it in a list
                timeStartTrial3Str = UserLookedAtNewTrial(userKeysWTime)
                timeStartTrial3Str1, t1, t2 = timeStartTrial3Str.partition('+')
                timeStartTrial3UserKeys = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStartTrial3Str1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                
                timeExptEndStr = FindTimeEnd(scratchPad)
                timeExptEndStr1, t1, t2 = timeExptEndStr.partition('+')
                timeExptEndScratchPad = (datetime.datetime.strptime(re.sub('[:.T]','-',timeExptEndStr1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                

                # call function to check when scratchpad is looked at and save it in a list
                scratchPadKeyTime = UserLookedAtScratchPad(userKeysWTime)

                pupilData = computeAggregateAverage(scratchPadKeyTime, timeStartTrial3UserKeys, timeExptEndScratchPad, time_filterFormatted, pupilLogL_filter, pupilLogR_filter, gazeLog, subjName)
                


akt_MS
KEA_MS
mcc_MS
MK_DT
ok_MS
pt_DT
sh_MS
hc_MS
ib_MS
km_DT
ma_DT
pgba_DT
smn_DT
jek_MS
lr_MS
mm_MS
snk_DT


In [27]:
if pupilData['DTFirst'] > 0 or pupilData['MSFirst'] > 0:
    #print(pupilData)
    # if the DT and MS data are of different sizes,
    pupilSizeMin = min(len(pupilData['DTLeft']), len(pupilData['MSLeft']))
    fig = plt.figure()
    axL = fig.add_subplot(2, 1, 1)
    axR = fig.add_subplot(2, 1, 2)
    xAxis = np.arange(0, float(pupilSizeMin/90), float(1/90))
    pupilPlot = dict()
    pupilPlot['DTLeft'] = [x/pupilData['DTNumber'] for x in pupilData['DTLeft']]
    pupilPlot['DTRight'] = [x/pupilData['DTNumber'] for x in pupilData['DTRight']]
    pupilPlot['MSLeft'] = [x/pupilData['MSNumber'] for x in pupilData['MSLeft']]
    pupilPlot['MSRight'] = [x/pupilData['MSNumber'] for x in pupilData['MSRight']]
    
    # Standard deviation
#     pupilPlot['DTLeftSquareRoot'] = [(math.fabs(pupilData['DTLeftSquared'][xInd]/pupilData['DTNumber']-(pupilPlot['DTLeft'][xInd])**2))**0.5 for xInd in range(0, len(pupilData['DTLeftSquared']))]
#     pupilPlot['DTRightSquareRoot'] = [(math.fabs(pupilData['DTRightSquared'][xInd]/pupilData['DTNumber']-(pupilPlot['DTRight'][xInd]))**2)**0.5 for xInd in range(0, len(pupilData['DTRightSquared']))]
#     pupilPlot['MSLeftSquareRoot'] = [(math.fabs(pupilData['MSLeftSquared'][xInd]/pupilData['MSNumber']-(pupilPlot['MSLeft'][xInd])**2))**0.5 for xInd in range(0, len(pupilData['MSLeftSquared']))]
#     pupilPlot['MSRightSquareRoot'] = [(math.fabs(pupilData['MSRightSquared'][xInd]/pupilData['MSNumber']-(pupilPlot['MSRight'][xInd])**2))**0.5 for xInd in range(0, len(pupilData['MSRightSquared']))]

    pupilPlot['DTLeftSquareRoot'] = [(math.fabs(((pupilData['DTNumber'])*(pupilData['DTLeftSquared'][xInd])-(pupilData['DTLeft'][xInd])**2)/((pupilData['DTNumber'])*(pupilData['DTNumber']-1))))**0.5 for xInd in range(0, len(pupilData['DTLeftSquared']))]
    pupilPlot['DTRightSquareRoot'] = [(math.fabs(((pupilData['DTNumber'])*(pupilData['DTRightSquared'][xInd])-(pupilData['DTRight'][xInd])**2)/((pupilData['DTNumber'])*(pupilData['DTNumber']-1))))**0.5 for xInd in range(0, len(pupilData['DTRightSquared']))]
    pupilPlot['MSLeftSquareRoot'] = [(math.fabs(((pupilData['MSNumber'])*(pupilData['MSLeftSquared'][xInd])-(pupilData['MSLeft'][xInd])**2)/((pupilData['MSNumber'])*(pupilData['MSNumber']-1))))**0.5 for xInd in range(0, len(pupilData['MSLeftSquared']))]
    pupilPlot['MSRightSquareRoot'] = [(math.fabs(((pupilData['MSNumber'])*(pupilData['MSRightSquared'][xInd])-(pupilData['MSRight'][xInd])**2)/((pupilData['MSNumber'])*(pupilData['MSNumber']-1))))**0.5 for xInd in range(0, len(pupilData['MSRightSquared']))]

    pupilPlot['DTLeftStdPositive'] = [pupilPlot['DTLeft'][xInd] + pupilPlot['DTLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['DTLeft']))]
    pupilPlot['DTLeftStdNegative'] = [pupilPlot['DTLeft'][xInd] - pupilPlot['DTLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['DTLeft']))]
    pupilPlot['MSLeftStdPositive'] = [pupilPlot['MSLeft'][xInd] + pupilPlot['MSLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['MSLeft']))]
    pupilPlot['MSLeftStdNegative'] = [pupilPlot['MSLeft'][xInd] - pupilPlot['MSLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['MSLeft']))]

    pupilPlot['DTRightStdPositive'] = [pupilPlot['DTRight'][xInd] + pupilPlot['DTRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['DTRight']))]
    pupilPlot['DTRightStdNegative'] = [pupilPlot['DTRight'][xInd] - pupilPlot['DTRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['DTRight']))]
    pupilPlot['MSRightStdPositive'] = [pupilPlot['MSRight'][xInd] + pupilPlot['MSRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['MSRight']))]
    pupilPlot['MSRightStdNegative'] = [pupilPlot['MSRight'][xInd] - pupilPlot['MSRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['MSRight']))]


    axL.plot(xAxis, pupilPlot['DTLeft'][0:pupilSizeMin], 'bo', label = 'Dwell Time')
    axL.plot(xAxis, pupilPlot['MSLeft'][0:pupilSizeMin], 'ro', label = 'Multi-key Selection')
    axL.fill_between(xAxis, pupilPlot['DTLeftStdPositive'][0:pupilSizeMin],  pupilPlot['DTLeftStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'blue')
    axL.fill_between(xAxis, pupilPlot['MSLeftStdPositive'][0:pupilSizeMin], pupilPlot['MSLeftStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'red')

    
    axL.set_title('Left pupil')
    axL.set_xlabel('Time [in s]')
    if absoluteSize:
        axL.set_ylabel('Absolute pupil size [in mm]')
    else:
        axL.set_ylabel('Relative pupil size')
    axL.legend()

    axR.plot(xAxis, pupilPlot['DTRight'][0:pupilSizeMin], 'bo', label = 'Dwell Time')
    axR.plot(xAxis, pupilPlot['MSRight'][0:pupilSizeMin], 'ro', label = 'Multi-key Selection')
    axR.fill_between(xAxis, pupilPlot['DTRightStdPositive'][0:pupilSizeMin], pupilPlot['DTRightStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'blue')
    axR.fill_between(xAxis, pupilPlot['MSRightStdPositive'][0:pupilSizeMin], pupilPlot['MSRightStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'red')
    axR.set_title('Right pupil')
    axR.legend()
    axR.set_xlabel('Time [in s]')
    if absoluteSize:
        axR.set_ylabel('Absolute pupil size [in mm]')
    else:
        axR.set_ylabel('Relative pupil size')


In [ ]:
pupilData['DTLeftMax']

In [ ]:
pupilData['DTLeftMin']

In [ ]:
pupilData['MSLeftMax']

In [ ]:
pupilData['MSLeftMin']